<a href="https://colab.research.google.com/github/BeatrixBlaine/DS-C1/blob/main/3_5_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# confidence

import numpy as np

def confidence_score(probabilities):
    """
    Confidence = 1 - normalized entropy
    """
    probs = np.clip(probabilities, 1e-12, 1.0)
    entropy = -np.sum(probs * np.log(probs))
    max_entropy = np.log(len(probs))
    return 1.0 - (entropy / max_entropy)


In [ ]:
# diversity

from sklearn.metrics.pairwise import cosine_similarity

def diversity_score(embedding, selected_embeddings):
    """
    Diversity = 1 - max cosine similarity
    """
    if len(selected_embeddings) == 0:
        return 1.0

    sims = cosine_similarity(
        embedding.reshape(1, -1),
        np.vstack(selected_embeddings)
    )
    return 1.0 - np.max(sims)


In [7]:
# Cognitive state tracker

class CognitiveState:
    def __init__(self):
        self.entropy_history = []
        self.confidence_history = []

    def update(self, probabilities):
        probs = np.clip(probabilities, 1e-12, 1.0)
        entropy = -np.sum(probs * np.log(probs))
        confidence = 1.0 - entropy / np.log(len(probs))

        self.entropy_history.append(entropy)
        self.confidence_history.append(confidence)

    def uncertainty_trend(self):
        """
        Measures cognitive saturation / confusion
        """
        if len(self.entropy_history) < 2:
            return 0.0
        return self.entropy_history[-1] - self.entropy_history[-2]

In [6]:
# cognitive state score

def cognitive_state_score(cognitive_state):
    """
    Higher score when uncertainty stabilizes
    """
    delta_entropy = cognitive_state.uncertainty_trend()

    # If entropy increases → cognitive overload
    if delta_entropy > 0:
        return -delta_entropy
    return abs(delta_entropy)


In [9]:
# csass scoring function

def csass_score(probabilities,
                embedding,
                selected_embeddings,
                cognitive_state,
                alpha=0.5,
                beta=0.3,
                gamma=0.2):
    """
    Cognitive-State Aware Scoring Function
    """
    conf = confidence_score(probabilities)
    div = diversity_score(embedding, selected_embeddings)
    cog = cognitive_state_score(cognitive_state)

    return alpha * conf + beta * div + gamma * cog


In [10]:
# sequential selection algorithm

def csass_selection(candidates, k=5,
                    alpha=0.5, beta=0.3, gamma=0.2):
    """
    candidates: list of dicts with
      - 'probs': np.array
      - 'embedding': np.array
    """
    selected = []
    selected_embeddings = []
    cognitive_state = CognitiveState()

    for t in range(k):
        scores = []

        for c in candidates:
            score = csass_score(
                c['probs'],
                c['embedding'],
                selected_embeddings,
                cognitive_state,
                alpha, beta, gamma
            )
            scores.append(score)

        best_idx = np.argmax(scores)
        best = candidates.pop(best_idx)

        selected.append(best)
        selected_embeddings.append(best['embedding'])

        # Update cognitive state AFTER selection
        cognitive_state.update(best['probs'])

    return selected
